In [1]:
# preprocessing.py

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy.spatial.transform import Rotation as R
import joblib
import gc
import os
import warnings

warnings.filterwarnings('ignore')

# ==============================================================================
#                            CONFIGURATION
# ==============================================================================
# --- File Paths ---
# Adjust these paths based on your project structure
DATA_DIR = '/kaggle/input/cmi-detect-behavior-with-sensor-data' # Example Kaggle path
TRAIN_CSV = os.path.join(DATA_DIR, 'train.csv')
TRAIN_DEMO_CSV = os.path.join(DATA_DIR, 'train_demographics.csv')

# --- Preprocessing Parameters ---
TOF_FILL_VALUE = 600 # Value to replace -1 in TOF

# --- Output Paths ---
OUTPUT_DIR = './data/preprocessed' # Save outputs here
SCALER_PATH = os.path.join(OUTPUT_DIR, 'standard_scaler.joblib')
LABEL_ENCODER_PATH = os.path.join(OUTPUT_DIR, 'label_encoder.joblib')
IMU_COLS_PATH = os.path.join(OUTPUT_DIR, 'imu_feature_cols_3branch.pkl')
THM_COLS_PATH = os.path.join(OUTPUT_DIR, 'thm_feature_cols_3branch.pkl')
TOF_COLS_PATH = os.path.join(OUTPUT_DIR, 'tof_feature_cols_3branch.pkl')
ALL_COLS_PATH = os.path.join(OUTPUT_DIR, 'all_feature_cols_3branch.pkl') # List of all scaled features
PROCESSED_TRAIN_PATH = os.path.join(OUTPUT_DIR, 'train_processed.parquet') # Save processed data

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory created/exists at: {OUTPUT_DIR}")


# ==============================================================================
#                 HELPER FUNCTIONS (Preprocessing Steps)
# ==============================================================================

# --- Define Sensor Columns ---
# Define these globally or pass them around as needed
acc_cols = ['acc_x', 'acc_y', 'acc_z']
rot_cols = ['rot_x', 'rot_y', 'rot_z', 'rot_w']
# Dynamically get thm/tof cols after loading
try:
    _temp_df = pd.read_csv(TRAIN_CSV, nrows=0) # Read only header
    thm_cols = sorted([col for col in _temp_df.columns if 'thm_' in col])
    tof_cols = sorted([col for col in _temp_df.columns if 'tof_' in col])
    del _temp_df
except Exception as e:
    print(f"Warning: Could not dynamically determine thm/tof columns from header: {e}")
    # Fallback to static definition if dynamic fails
    thm_cols = ['thm_1', 'thm_2', 'thm_3', 'thm_4', 'thm_5']
    tof_cols = sorted([f'tof_{s}_v{i}' for s in range(1, 6) for i in range(64)])


imu_cols = acc_cols + rot_cols
non_imu_sensor_cols = thm_cols + tof_cols
all_initial_sensor_cols = imu_cols + non_imu_sensor_cols

TARGET_GESTURES = [
    'Above ear - pull hair', 'Cheek - pinch skin', 'Eyebrow - pull hair',
    'Eyelash - pull hair', 'Forehead - pull hairline', 'Forehead - scratch',
    'Neck - pinch skin', 'Neck - scratch'
]

def preprocess_impute(df, tof_fill_value=600):
    """Handles imputation for sensor data."""
    print(f"Imputing with TOF fill value: {tof_fill_value}...")
    # Ensure columns exist before modifying
    present_tof_cols = [col for col in tof_cols if col in df.columns]
    present_thm_cols = [col for col in thm_cols if col in df.columns]
    present_all_initial_cols = [col for col in all_initial_sensor_cols if col in df.columns]

    if present_tof_cols:
        df[present_tof_cols] = df[present_tof_cols].replace(-1, tof_fill_value)
    if present_thm_cols:
        for col in present_thm_cols:
             df[col] = df[col].apply(lambda x: np.nan if x < 20 else x)
        if 'sequence_id' in df.columns:
            # Use transform for interpolation within groups
            # Add group_keys=False to avoid potential multi-index issues later
            df[present_thm_cols] = df.groupby('sequence_id', group_keys=False)[present_thm_cols].transform(
                lambda x: x.interpolate(method='linear', limit_direction='both', axis=0)
            )
        else: # Single sequence case
            df[present_thm_cols] = df[present_thm_cols].interpolate(method='linear', limit_direction='both', axis=0)

    group_cols_to_fill = list(set(present_all_initial_cols) - set(present_thm_cols))
    if 'sequence_id' in df.columns and group_cols_to_fill:
         df[group_cols_to_fill] = df.groupby('sequence_id', group_keys=False)[group_cols_to_fill].transform(
             lambda x: x.ffill().bfill()
         )
    elif group_cols_to_fill: # Single sequence case
         df[group_cols_to_fill] = df[group_cols_to_fill].ffill().bfill()

    # Fill any remaining NaNs globally AFTER group operations
    if present_all_initial_cols:
        df[present_all_initial_cols] = df[present_all_initial_cols].fillna(0)
    print("Imputation complete.")
    return df

def correct_handedness(df, demo_df):
    """Corrects sensor readings based on subject handedness."""
    print("Correcting handedness...")
    if 'handedness' not in df.columns:
      if 'subject' in df.columns and 'subject' in demo_df.columns:
          # Ensure subject columns are compatible type before merge
          df['subject'] = df['subject'].astype(str)
          demo_df['subject'] = demo_df['subject'].astype(str)
          df = df.merge(demo_df[['subject', 'handedness']], on='subject', how='left')
          # Handle potential NaNs in handedness after merge (e.g., test subjects not in demo)
          df['handedness'] = df['handedness'].fillna('Right')
      else:
          df['handedness'] = 'Right' # Fallback if subject info missing

    left_handed_mask = df['handedness'] == 'Left'
    if left_handed_mask.any():
        if 'acc_x' in df.columns: df.loc[left_handed_mask, 'acc_x'] *= -1
        if 'rot_y' in df.columns: df.loc[left_handed_mask, 'rot_y'] *= -1
        if 'rot_z' in df.columns: df.loc[left_handed_mask, 'rot_z'] *= -1
        # Add TOF/THM mirroring here if implementing
    print("Handedness correction complete.")
    return df

def correct_upside_down(df):
    """Corrects sensor readings for known upside-down subjects."""
    print("Correcting upside-down subjects...")
    upside_down_subjects = ['SUBJ_019262', 'SUBJ_045235']
    if 'subject' in df.columns:
        df['subject'] = df['subject'].astype(str) # Ensure subject is string
        ud_mask = df['subject'].isin(upside_down_subjects)
        if ud_mask.any():
            if all(c in df.columns for c in ['acc_x', 'acc_y']): df.loc[ud_mask, ['acc_x', 'acc_y']] *= -1
            if all(c in df.columns for c in ['rot_x', 'rot_y']): df.loc[ud_mask, ['rot_x', 'rot_y']] *= -1
            # Add TOF/THM mirroring here if implementing
    print("Upside-down correction complete.")
    return df

def add_linear_acceleration(df):
    """Calculates linear acceleration by removing gravity."""
    print("Calculating linear acceleration...")
    if not all(col in df.columns for col in rot_cols + acc_cols):
        print("Warning: Acc/Rot columns missing. Assigning raw acceleration to linear.")
        # Ensure target columns exist before assignment
        present_acc_cols = [c for c in acc_cols if c in df.columns]
        lin_acc_map = {'acc_x': 'lin_acc_x', 'acc_y': 'lin_acc_y', 'acc_z': 'lin_acc_z'}
        for col in acc_cols:
             df[lin_acc_map[col]] = df[col] if col in df.columns else 0.0
        return df

    # Ensure float type before processing
    for col in rot_cols + acc_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    quats = df[rot_cols].values
    accels = df[acc_cols].values
    linear_accel = np.zeros_like(accels)
    gravity_world = np.array([0, 0, 1.0]) # Z-up, magnitude 1

    quats_numeric = np.nan_to_num(quats) # Replace NaN with 0 for checks
    # Check for non-zero quaternions and approximate unit norm
    valid_quat_mask = ~np.all(quats_numeric == 0, axis=1) & (np.abs(np.linalg.norm(quats_numeric, axis=1) - 1.0) < 1e-2) # Adjusted tolerance
    valid_quats_data = quats_numeric[valid_quat_mask]

    if len(valid_quats_data) > 0:
        try:
            # Rigorous normalization
            norms = np.linalg.norm(valid_quats_data, axis=1, keepdims=True)
            # Prevent division by zero or very small norms
            norms[norms < 1e-6] = 1.0
            valid_quats_normalized = valid_quats_data / norms
            # Clip to ensure values are within valid range for R.from_quat
            valid_quats_normalized = np.clip(valid_quats_normalized, -1.0, 1.0)

            r = R.from_quat(valid_quats_normalized)
            r_inv = r.inv()
            gravity_sensor_frame = r_inv.apply(gravity_world)
            linear_accel[valid_quat_mask] = accels[valid_quat_mask] - gravity_sensor_frame
        except Exception as e:
            print(f"Warning: Scipy Rotation error during gravity removal: {e}. Using raw accel for some rows.")
            linear_accel[valid_quat_mask] = accels[valid_quat_mask] # Fallback

    invalid_quat_mask = ~valid_quat_mask
    linear_accel[invalid_quat_mask] = accels[invalid_quat_mask] # Use raw accel if quat invalid

    df['lin_acc_x'], df['lin_acc_y'], df['lin_acc_z'] = linear_accel[:, 0], linear_accel[:, 1], linear_accel[:, 2]
    print("Linear acceleration added.")
    return df

def add_basic_features(df):
    """Adds magnitude features."""
    print("Adding basic features (magnitudes)...")
     # Ensure float type
    acc_mag_cols = ['acc_x', 'acc_y', 'acc_z']
    lin_acc_mag_cols = ['lin_acc_x', 'lin_acc_y', 'lin_acc_z']
    rot_mag_cols = ['rot_x', 'rot_y', 'rot_z']
    for col in acc_mag_cols + lin_acc_mag_cols + rot_mag_cols:
         if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    if all(col in df.columns for col in acc_mag_cols):
        df['acc_mag'] = np.linalg.norm(df[acc_mag_cols].values, axis=1)
    else: df['acc_mag'] = 0.0

    if all(col in df.columns for col in lin_acc_mag_cols):
        df['lin_acc_mag'] = np.linalg.norm(df[lin_acc_mag_cols].values, axis=1)
    else: df['lin_acc_mag'] = 0.0

    if all(col in df.columns for col in rot_mag_cols):
        df['rot_mag'] = np.linalg.norm(df[rot_mag_cols].values, axis=1)
    else: df['rot_mag'] = 0.0
    print("Basic features added.")
    return df

def full_preprocess_pipeline(df, demo_df, scaler=None, fit_scaler=False):
    """Applies the full preprocessing pipeline using CPU libraries."""
    print("-" * 30)
    print("Starting Full Preprocessing Pipeline...")
    # Ensure IDs are numeric/string
    if 'sequence_id' in df.columns and df['sequence_id'].dtype == 'object':
         df['sequence_id'] = df['sequence_id'].str.extract('(\d+)').astype(int)
    if 'subject' in df.columns: df['subject'] = df['subject'].astype(str)
    if 'subject' in demo_df.columns: demo_df['subject'] = demo_df['subject'].astype(str)

    # --- Preprocessing Steps ---
    df_processed = preprocess_impute(df.copy(), tof_fill_value=TOF_FILL_VALUE)
    df_processed = correct_handedness(df_processed, demo_df)
    df_processed = correct_upside_down(df_processed)
    df_processed = add_linear_acceleration(df_processed)
    df_processed = add_basic_features(df_processed)

    # --- Scaling ---
    engineered_imu_cols = ['lin_acc_x', 'lin_acc_y', 'lin_acc_z', 'acc_mag', 'lin_acc_mag', 'rot_mag']
    potential_cols_to_scale = list(set(imu_cols + non_imu_sensor_cols + engineered_imu_cols))
    # Ensure columns exist and sort for consistent order
    cols_to_scale = sorted([col for col in potential_cols_to_scale if col in df_processed.columns])

    if not cols_to_scale:
         print("Warning: No columns identified for scaling.")
         # Add feature_names_in_ attribute if scaler exists but no columns to scale
         if scaler is not None and not hasattr(scaler, 'feature_names_in_'):
             scaler.feature_names_in_ = []
         return df_processed, scaler, cols_to_scale # Return early if no cols

    if fit_scaler:
        print(f"Fitting StandardScaler on {len(cols_to_scale)} features.")
        scaler = StandardScaler()
        # Ensure data is numeric before fitting
        df_processed[cols_to_scale] = df_processed[cols_to_scale].apply(pd.to_numeric, errors='coerce').fillna(0)
        df_processed[cols_to_scale] = scaler.fit_transform(df_processed[cols_to_scale])
        # Store feature names IN THE ORDER THEY WERE FITTED
        scaler.feature_names_in_ = cols_to_scale
        print("Scaler fitted.")
    elif scaler is not None:
        print(f"Transforming features using loaded scaler...")
        # Ensure scaler has feature names
        if not hasattr(scaler, 'feature_names_in_') or scaler.feature_names_in_ is None:
             raise ValueError("Loaded scaler is missing 'feature_names_in_'. Cannot proceed.")

        # Ensure columns match scaler's expectations
        cols_available_ordered = [col for col in scaler.feature_names_in_ if col in df_processed.columns]
        # Create a DataFrame with the exact columns and order the scaler expects
        df_ordered = pd.DataFrame(0.0, index=df_processed.index, columns=scaler.feature_names_in_)
        # Fill with available data, respecting scaler's column order
        if cols_available_ordered: # Check if there are any columns to fill
            df_ordered[cols_available_ordered] = df_processed[cols_available_ordered]
        else:
             print("Warning: No columns from scaler found in the dataframe to transform.")

        if len(cols_available_ordered) < len(scaler.feature_names_in_):
            missing_cols = list(set(scaler.feature_names_in_) - set(cols_available_ordered))
            print(f"Warning: Filled {len(missing_cols)} missing columns with zeros before scaling: {missing_cols}")
        # Ensure data is numeric before transforming
        df_ordered[scaler.feature_names_in_] = df_ordered[scaler.feature_names_in_].apply(pd.to_numeric, errors='coerce').fillna(0)
        # Transform using the reordered/filled dataframe
        df_processed[scaler.feature_names_in_] = scaler.transform(df_ordered[scaler.feature_names_in_])
        print("Transformation complete.")
    else:
        raise ValueError("Scaler must be provided if fit_scaler is False")

    # Drop handedness column after scaling (if it wasn't used as feature)
    if 'handedness' in df_processed.columns and 'handedness' not in cols_to_scale:
        df_processed = df_processed.drop(columns=['handedness'])

    print("Full Preprocessing Pipeline Finished.")
    print("-" * 30)
    return df_processed, scaler, cols_to_scale

# ==============================================================================
#                      MAIN PREPROCESSING EXECUTION
# ==============================================================================
print("Running Main Preprocessing Script...")

# --- Load Data ---
print("Loading data...")
try:
    train_df_main = pd.read_csv(TRAIN_CSV)
    train_demo_df_main = pd.read_csv(TRAIN_DEMO_CSV)
    print(f"Loaded Train shape: {train_df_main.shape}")
except FileNotFoundError:
    print(f"Error: Training files not found at {DATA_DIR}. Exiting.")
    exit()
except Exception as e:
    print(f"Error loading data: {e}. Exiting.")
    exit()

# --- Apply Full Preprocessing to Training Data ---
train_processed_df_main, scaler_main, feature_columns_main = full_preprocess_pipeline(
    train_df_main, train_demo_df_main, scaler=None, fit_scaler=True
)
print(f"Processed Train shape: {train_processed_df_main.shape}")
print(f"Number of features scaled: {len(feature_columns_main)}")

# --- Define Feature Lists for Branches (Example) ---
imu_eng_cols_main = ['lin_acc_x', 'lin_acc_y', 'lin_acc_z', 'acc_mag', 'lin_acc_mag', 'rot_mag']
model_imu_feature_cols_main = sorted([col for col in feature_columns_main if col in imu_cols + imu_eng_cols_main])
model_thm_feature_cols_main = sorted([col for col in feature_columns_main if col in thm_cols])
model_tof_feature_cols_main = sorted([col for col in feature_columns_main if col in tof_cols])

N_FEATURES_IMU_main = len(model_imu_feature_cols_main)
N_FEATURES_THM_main = len(model_thm_feature_cols_main)
N_FEATURES_TOF_main = len(model_tof_feature_cols_main)
print(f"Branch Features: IMU={N_FEATURES_IMU_main}, THM={N_FEATURES_THM_main}, TOF={N_FEATURES_TOF_main}")


# --- Save Objects ---
print("\nSaving preprocessor objects...")
joblib.dump(scaler_main, SCALER_PATH)
joblib.dump(model_imu_feature_cols_main, IMU_COLS_PATH)
joblib.dump(model_thm_feature_cols_main, THM_COLS_PATH)
joblib.dump(model_tof_feature_cols_main, TOF_COLS_PATH)
joblib.dump(feature_columns_main, ALL_COLS_PATH) # Save list of all scaled features
print(f"Objects saved to: {OUTPUT_DIR}")

# --- Label Encode and Save Encoder ---
print("Encoding target and saving encoder...")
label_encoder_main = LabelEncoder()
if 'gesture' in train_processed_df_main.columns:
    train_processed_df_main['gesture_encoded'] = label_encoder_main.fit_transform(train_processed_df_main['gesture'])
    joblib.dump(label_encoder_main, LABEL_ENCODER_PATH)
    print(f"Target Classes ({len(label_encoder_main.classes_)}): {label_encoder_main.classes_}")
    print(f"Label encoder saved to: {LABEL_ENCODER_PATH}")
else:
    print("Warning: 'gesture' column not found in processed data. Cannot save label encoder.")

# --- Save Processed Data (Optional) ---
print(f"\nSaving processed training data to {PROCESSED_TRAIN_PATH}...")
try:
    # Ensure directory exists before saving
    os.makedirs(os.path.dirname(PROCESSED_TRAIN_PATH), exist_ok=True)
    # Ensure correct types before saving to parquet
    df_to_save = train_processed_df_main.copy()
    for col in df_to_save.select_dtypes(include='object').columns:
        # Check if column still exists before converting
        if col in df_to_save.columns:
            df_to_save[col] = df_to_save[col].astype(str) # Convert object cols to string

    df_to_save.to_parquet(PROCESSED_TRAIN_PATH, index=False)
    print("Processed data saved.")
except Exception as e:
    print(f"Error saving processed data: {e}")

print("\nPreprocessing script finished.")

# Clean up large dataframes from memory
del train_df_main, train_demo_df_main, train_processed_df_main
gc.collect()

Output directory created/exists at: ./data/preprocessed
Running Main Preprocessing Script...
Loading data...
Loaded Train shape: (574945, 341)
------------------------------
Starting Full Preprocessing Pipeline...
Imputing with TOF fill value: 600...
Imputation complete.
Correcting handedness...
Handedness correction complete.
Correcting upside-down subjects...
Upside-down correction complete.
Calculating linear acceleration...
Linear acceleration added.
Adding basic features (magnitudes)...
Basic features added.
Fitting StandardScaler on 338 features.
Scaler fitted.
Full Preprocessing Pipeline Finished.
------------------------------
Processed Train shape: (574945, 347)
Number of features scaled: 338
Branch Features: IMU=13, THM=5, TOF=320

Saving preprocessor objects...
Objects saved to: ./data/preprocessed
Encoding target and saving encoder...
Target Classes (18): ['Above ear - pull hair' 'Cheek - pinch skin' 'Drink from bottle/cup'
 'Eyebrow - pull hair' 'Eyelash - pull hair'
 'Fee

0

In [2]:
5

5

In [3]:
# data_preparation.py

import pandas as pd
import numpy as np
import joblib
import os
from tqdm import tqdm # For progress bar
import gc
import warnings

warnings.filterwarnings('ignore')

# ==============================================================================
#                            CONFIGURATION
# ==============================================================================
# --- Input Paths ---
# Should match the OUTPUT_DIR and filenames from the *preprocessing script*
PREPROCESSED_DIR = './data/preprocessed' # Directory where preprocessed files ARE SAVED
PROCESSED_TRAIN_PATH = os.path.join(PREPROCESSED_DIR, 'train_processed.parquet')
ALL_COLS_PATH = os.path.join(PREPROCESSED_DIR, 'all_feature_cols_3branch.pkl') # List of all scaled columns
LABEL_ENCODER_PATH = os.path.join(PREPROCESSED_DIR, 'label_encoder.joblib') # Load encoder to ensure y-values are valid

# --- Model Input Parameters ---
MAX_LENGTH = 192 # Sequence length for padding/truncating

# --- Output Paths ---
# Where to SAVE the final NumPy arrays for the model
OUTPUT_NP_DIR = './data/model_input'
X_PATH = os.path.join(OUTPUT_NP_DIR, 'X_train.npy')
Y_PATH = os.path.join(OUTPUT_NP_DIR, 'y_train.npy')
GROUPS_PATH = os.path.join(OUTPUT_NP_DIR, 'groups_train.npy')
SEQ_IDS_PATH = os.path.join(OUTPUT_NP_DIR, 'train_seq_ids.npy') # Path to save sequence IDs

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_NP_DIR, exist_ok=True)
print(f"NumPy output directory created/exists at: {OUTPUT_NP_DIR}")


# ==============================================================================
#                      LOAD PREPROCESSED DATA & OBJECTS
# ==============================================================================
print("Loading preprocessed data and feature lists...")
try:
    train_processed_df = pd.read_parquet(PROCESSED_TRAIN_PATH)
    # Load the list of all columns that were included after preprocessing
    final_column_names = joblib.load(ALL_COLS_PATH)
    # Load Label Encoder to validate labels later if needed
    label_encoder = joblib.load(LABEL_ENCODER_PATH)
    N_CLASSES = len(label_encoder.classes_) # Get number of classes

    # --- Derive feature_cols from the loaded list ---
    # Define columns that are NOT features
    non_feature_cols = [
        'sequence_id', 'subject', 'gesture', 'row_id', 'sequence_counter',
        'orientation', 'behavior', 'phase', 'sequence_type', 'handedness', # If kept during preprocessing
        'gesture_encoded' # The target label
    ]
    # Filter final_column_names to get only feature columns, ensure they exist
    feature_cols = sorted([
        col for col in final_column_names
        if col not in non_feature_cols and col in train_processed_df.columns
    ])

    print(f"Loaded processed data with shape: {train_processed_df.shape}")
    print(f"Identified {len(feature_cols)} feature columns to use.")
    if len(feature_cols) == 0:
        raise ValueError("No feature columns identified after filtering. Check ALL_COLS_PATH and non_feature_cols.")
    # print("Feature columns:", feature_cols) # Uncomment to verify features

    # Verify essential columns exist
    if 'sequence_id' not in train_processed_df.columns: raise ValueError("Missing 'sequence_id'")
    if 'gesture_encoded' not in train_processed_df.columns: raise ValueError("Missing 'gesture_encoded'")
    if 'subject' not in train_processed_df.columns: raise ValueError("Missing 'subject'")

except FileNotFoundError as e:
    print(f"Error: Required file not found. Ensure preprocessing script ran successfully and saved outputs to '{PREPROCESSED_DIR}'. Missing: {e.filename}")
    exit()
except Exception as e:
    print(f"An error occurred during loading: {e}")
    exit()

# ==============================================================================
#                      PREPARE MODEL INPUT ARRAYS
# ==============================================================================
print("\nPreparing model input arrays (Grouping, Padding/Truncating)...")

all_sequences = []
all_labels = []
all_groups = []
all_sequence_ids = [] # List to store sequence IDs
num_features = len(feature_cols) # Use the derived feature_cols list

# Group by sequence ID
grouped_data = train_processed_df.groupby('sequence_id')
total_sequences = len(grouped_data)
print(f"Processing {total_sequences} sequences...")
skipped_sequences_prep = 0

# Iterate through each sequence group with a progress bar
for name, group in tqdm(grouped_data, total=total_sequences, desc="Processing Sequences"):
    # Extract features, label, and group
    try:
        # Check for NaNs *before* accessing values
        if group[feature_cols].isnull().values.any():
             # print(f"Warning: Sequence {name} contains NaN values. Skipping.") # Can be verbose
             skipped_sequences_prep += 1
             continue

        sequence_features = group[feature_cols].values.astype(np.float32) # Ensure float32
        label = group['gesture_encoded'].iloc[0] # Get the single label for the sequence
        subject = group['subject'].iloc[0]       # Get the subject ID

        current_length = sequence_features.shape[0]

        if current_length == 0:
            # print(f"Warning: Sequence {name} is empty after filtering NaNs. Skipping.") # Can be verbose
            skipped_sequences_prep += 1
            continue # Skip empty sequences

        # Validate label is within expected range
        if not (0 <= label < N_CLASSES):
            # print(f"Warning: Sequence {name} has invalid label {label}. Skipping.")
            skipped_sequences_prep += 1
            continue

        padded_sequence = np.zeros((MAX_LENGTH, num_features), dtype=np.float32) # Initialize with zeros

        # Pad or Truncate (using pre-padding)
        if current_length >= MAX_LENGTH:
            # Truncate: Take the last MAX_LENGTH steps
            padded_sequence = sequence_features[-MAX_LENGTH:]
        else:
            # Pad: Add zeros at the beginning (pre-padding)
            pad_width = MAX_LENGTH - current_length
            padded_sequence[pad_width:] = sequence_features

        # Append results to lists
        all_sequences.append(padded_sequence)
        all_labels.append(label)
        all_groups.append(subject)
        all_sequence_ids.append(name) # STORE SEQUENCE ID

    except Exception as e:
        print(f"Error processing sequence {name}: {e}. Skipping.")
        skipped_sequences_prep += 1
        continue

print(f"Skipped {skipped_sequences_prep} sequences during preparation due to issues.")

# Convert lists to NumPy arrays
if not all_sequences:
    print("FATAL ERROR: No valid sequences were processed. Cannot create NumPy arrays.")
    exit()

X = np.array(all_sequences)
y = np.array(all_labels)
groups = np.array(all_groups)
train_seq_ids = np.array(all_sequence_ids) # CONVERT SEQ IDS TO NUMPY ARRAY


print("\nModel input preparation complete.")
print(f"Shape of X: {X.shape}") # Should be (num_sequences, MAX_LENGTH, num_features)
print(f"Shape of y: {y.shape}")   # Should be (num_sequences,)
print(f"Shape of groups: {groups.shape}") # Should be (num_sequences,)
print(f"Shape of train_seq_ids: {train_seq_ids.shape}")

# ==============================================================================
#                      SAVE MODEL INPUT ARRAYS
# ==============================================================================
print(f"\nSaving prepared NumPy arrays to {OUTPUT_NP_DIR}...")
try:
    np.save(X_PATH, X)
    np.save(Y_PATH, y)
    np.save(GROUPS_PATH, groups)
    np.save(SEQ_IDS_PATH, train_seq_ids) # SAVE SEQUENCE IDS
    print("Arrays saved successfully.")
except Exception as e:
    print(f"Error saving NumPy arrays: {e}")

print("\nData Preparation script finished. You now have X, y, groups, and train_seq_ids ready for model training.")

# Clean up memory
del train_processed_df, X, y, groups, train_seq_ids, all_sequences, all_labels, all_groups, all_sequence_ids
gc.collect()

NumPy output directory created/exists at: ./data/model_input
Loading preprocessed data and feature lists...
Loaded processed data with shape: (574945, 348)
Identified 338 feature columns to use.

Preparing model input arrays (Grouping, Padding/Truncating)...
Processing 8151 sequences...


Processing Sequences: 100%|██████████| 8151/8151 [00:11<00:00, 737.75it/s]


Skipped 0 sequences during preparation due to issues.

Model input preparation complete.
Shape of X: (8151, 192, 338)
Shape of y: (8151,)
Shape of groups: (8151,)
Shape of train_seq_ids: (8151,)

Saving prepared NumPy arrays to ./data/model_input...
Arrays saved successfully.

Data Preparation script finished. You now have X, y, groups, and train_seq_ids ready for model training.


0

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from scipy.optimize import linear_sum_assignment
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import joblib
import gc
import os
import warnings

warnings.filterwarnings('ignore')

print("TensorFlow Version:", tf.__version__)

# ==============================================================================
#                            CONFIGURATION
# ==============================================================================
N_SPLITS = 2
BATCH_SIZE = 128 # Adjust based on memory
EPOCHS = 50 # Adjust based on validation performance
MAX_LENGTH = 192 # Must match the MAX_LENGTH used in data preparation
SEED = 42
SENSOR_DROPOUT_RATE = 0.5 # Probability of dropping non-IMU sensors during training

# Dropout Rates for Model
CNN_SPATIAL_DROPOUT_RATE = 0.25
LSTM_DROPOUT_RATE = 0.4 # Using GRU, applied to GRU
LSTM_RECURRENT_DROPOUT_RATE = 0.4 # Applied to GRU
DENSE_DROPOUT_RATE = 0.5

np.random.seed(SEED)
tf.random.set_seed(SEED)

# Paths for LOADING objects and prepared arrays
PREPROCESSED_DIR = './data/preprocessed'
LABEL_ENCODER_PATH = os.path.join(PREPROCESSED_DIR, 'label_encoder.joblib')
ALL_COLS_PATH = os.path.join(PREPROCESSED_DIR, 'all_feature_cols_3branch.pkl') # List of all columns in X
IMU_COLS_PATH = os.path.join(PREPROCESSED_DIR, 'imu_feature_cols_3branch.pkl')
THM_COLS_PATH = os.path.join(PREPROCESSED_DIR, 'thm_feature_cols_3branch.pkl')
TOF_COLS_PATH = os.path.join(PREPROCESSED_DIR, 'tof_feature_cols_3branch.pkl')

NUMPY_INPUT_DIR = './data/model_input' # Where X, y, groups were saved
X_PATH = os.path.join(NUMPY_INPUT_DIR, 'X_train.npy')
Y_PATH = os.path.join(NUMPY_INPUT_DIR, 'y_train.npy')
GROUPS_PATH = os.path.join(NUMPY_INPUT_DIR, 'groups_train.npy')
SEQ_IDS_PATH = os.path.join(NUMPY_INPUT_DIR, 'train_seq_ids.npy') # Load if needed for PP

# Paths for saving NEW models/results
OUTPUT_DIR = './data/3branch_dropout_output' # New directory for this model run
MODEL_TMPL = os.path.join(OUTPUT_DIR, "model_3branch_dropout_fold_{fold}.keras")
OOF_CSV_PATH = os.path.join(OUTPUT_DIR, "oof_predictions_3branch_dropout.csv")

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Target Gestures ---
TARGET_GESTURES = [
    'Above ear - pull hair', 'Cheek - pinch skin', 'Eyebrow - pull hair',
    'Eyelash - pull hair', 'Forehead - pull hairline', 'Forehead - scratch',
    'Neck - pinch skin', 'Neck - scratch'
]
# TOF Reshaping dimensions
TOF_SENSORS = 5
TOF_GRID_DIM = 8

# ==============================================================================
#               LOAD PREPARED DATA & PREPROCESSING OBJECTS
# ==============================================================================
print("Loading pre-trained objects and prepared NumPy arrays...")
try:
    label_encoder = joblib.load(LABEL_ENCODER_PATH)
    all_feature_cols = joblib.load(ALL_COLS_PATH) # List of columns in X, in order
    model_imu_feature_cols = joblib.load(IMU_COLS_PATH)
    model_thm_feature_cols = joblib.load(THM_COLS_PATH)
    model_tof_feature_cols = joblib.load(TOF_COLS_PATH)

    N_CLASSES = len(label_encoder.classes_)
    N_FEATURES_ALL = len(all_feature_cols) # Total features in X

    # Load NumPy arrays
    X = np.load(X_PATH)
    y = np.load(Y_PATH)
    groups = np.load(GROUPS_PATH) # Subject IDs
    train_seq_ids = np.load(SEQ_IDS_PATH) # Load sequence IDs if saved and needed

    # --- Find indices for splitting X into branches ---
    # Create mapping from column name to index in the full X array
    col_to_idx = {col: idx for idx, col in enumerate(all_feature_cols)}
    imu_indices = sorted([col_to_idx[col] for col in model_imu_feature_cols if col in col_to_idx])
    thm_indices = sorted([col_to_idx[col] for col in model_thm_feature_cols if col in col_to_idx])
    tof_indices = sorted([col_to_idx[col] for col in model_tof_feature_cols if col in col_to_idx])

    N_FEATURES_IMU = len(imu_indices)
    N_FEATURES_THM = len(thm_indices)
    N_FEATURES_TOF = len(tof_indices) # Should be 320

    # Calculate scaled missing value indicator (Scaled Zero) - requires scaler
    # Since scaler wasn't loaded here, assume 0.0 is the padding value
    # If using StandardScaler and filled NaNs with 0 before scaling, this is likely correct
    MISSING_NON_IMU_SCALED_VALUE = 0.0
    print(f"Using assumed scaled missing non-IMU indicator value: {MISSING_NON_IMU_SCALED_VALUE:.4f}")

    print("Loaded NumPy arrays and found feature indices:")
    print(f"X shape: {X.shape} (Expected features: {N_FEATURES_ALL})")
    print(f"y shape: {y.shape}, groups shape: {groups.shape}")
    print(f"IMU features: {N_FEATURES_IMU}, THM features: {N_FEATURES_THM}, TOF features: {N_FEATURES_TOF}")
    print(f"Number of classes: {N_CLASSES}")

    # Basic shape validation
    if X.shape[0] != len(y) or X.shape[0] != len(groups):
        raise ValueError("Loaded X, y, and groups have inconsistent lengths.")
    if X.shape[2] != N_FEATURES_ALL:
        raise ValueError(f"Loaded X feature dimension ({X.shape[2]}) doesn't match expected ({N_FEATURES_ALL}).")
    if N_FEATURES_TOF != TOF_SENSORS * TOF_GRID_DIM * TOF_GRID_DIM:
         print(f"Warning: TOF feature count ({N_FEATURES_TOF}) mismatch expected ({TOF_SENSORS*TOF_GRID_DIM**2}).")


except FileNotFoundError as e:
    print(f"FATAL ERROR: Required file not found. Ensure previous script saved arrays/objects. Missing: {e.filename}")
    exit()
except Exception as e:
    print(f"FATAL ERROR loading data or objects: {e}")
    exit()


# ==============================================================================
#      SPLIT X INTO BRANCHES & PREPARE FOR TF.DATA
# ==============================================================================
print("\nSplitting X into IMU, THM, TOF branches...")
X_imu = X[:, :, imu_indices]
X_thm = X[:, :, thm_indices]
X_tof_flat = X[:, :, tof_indices] # Shape (n_seq, max_len, 320)

# Reshape TOF now for the tf.data pipeline input
X_tof = X_tof_flat.reshape((-1, MAX_LENGTH, TOF_GRID_DIM, TOF_GRID_DIM, TOF_SENSORS))
print(f"Split array shapes: X_imu={X_imu.shape}, X_thm={X_thm.shape}, X_tof={X_tof.shape}")

del X, X_tof_flat # Free memory of the large combined array
gc.collect()

# ==============================================================================
#                 tf.data Pipeline with Sensor Dropout
# ==============================================================================

def sensor_dropout_tf_fn(features_dict, label):
    """Applies sensor dropout augmentation to THM and TOF tensors."""
    imu_feat = features_dict['imu_input']
    thm_feat = features_dict['thm_input']
    tof_feat = features_dict['tof_input']

    if tf.random.uniform(()) < SENSOR_DROPOUT_RATE:
        # Replace THM and TOF features with the scaled missing value (zeros)
        thm_feat = tf.zeros_like(thm_feat) # Use zeros matching padding value
        tof_feat = tf.zeros_like(tof_feat) # Use zeros matching padding value

    # Return updated dictionary
    return {'imu_input': imu_feat, 'thm_input': thm_feat, 'tof_input': tof_feat}, label


def create_tf_dataset_from_np(X_imu_np, X_thm_np, X_tof_np, y_np, batch_size, shuffle=False, augment=False):
    """Creates a tf.data.Dataset directly from NumPy arrays for 3 branches."""
    # Create input dictionary from NumPy slices
    input_dict = {'imu_input': X_imu_np, 'thm_input': X_thm_np, 'tof_input': X_tof_np}

    # Create dataset from slices
    dataset = tf.data.Dataset.from_tensor_slices((input_dict, y_np))

    if shuffle:
        dataset = dataset.shuffle(len(y_np), seed=SEED, reshuffle_each_iteration=True)

    if augment:
        dataset = dataset.map(sensor_dropout_tf_fn, num_parallel_calls=tf.data.AUTOTUNE)

    # Batch the dataset - Padding is already done, so just batch
    dataset = dataset.batch(batch_size)

    # Prefetch for performance
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# ==============================================================================
#                      MODEL ARCHITECTURE (Shallow & Wide 3-Branch)
# ==============================================================================

def build_shallow_wide_3branch(seq_len, n_features_imu, n_features_thm, tof_shape, n_classes):
    """Defines the shallow-wide Three-Branch CNN-GRU model."""
    # Inputs
    input_imu = keras.Input(shape=(seq_len, n_features_imu), name='imu_input')
    input_thm = keras.Input(shape=(seq_len, n_features_thm), name='thm_input')
    input_tof = keras.Input(shape=(seq_len,) + tof_shape, name='tof_input') # tof_shape = (8, 8, 5)

    # --- IMU Branch (Wider 1D CNN) ---
    x_imu = layers.Masking(mask_value=0.0)(input_imu) # Mask padding zeros
    x_imu = layers.Conv1D(256, kernel_size=7, padding='same', activation='relu')(x_imu); x_imu = layers.BatchNormalization()(x_imu); x_imu = layers.SpatialDropout1D(CNN_SPATIAL_DROPOUT_RATE)(x_imu)
    pool_size_1d = 4; x_imu = layers.MaxPooling1D(pool_size=pool_size_1d)(x_imu)

    # --- THM Branch (Wider 1D CNN) ---
    x_thm = layers.Masking(mask_value=0.0)(input_thm) # Mask padding zeros
    x_thm = layers.Conv1D(128, kernel_size=7, padding='same', activation='relu')(x_thm); x_thm = layers.BatchNormalization()(x_thm); x_thm = layers.SpatialDropout1D(CNN_SPATIAL_DROPOUT_RATE)(x_thm)
    x_thm = layers.MaxPooling1D(pool_size=pool_size_1d)(x_thm)

    # --- TOF Branch (Wider 3D CNN) ---
    mask_tof = layers.Masking(mask_value=0.0)(input_tof) # Mask padding zeros
    x_tof = layers.Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(mask_tof); x_tof = layers.BatchNormalization()(x_tof); x_tof = layers.SpatialDropout3D(CNN_SPATIAL_DROPOUT_RATE)(x_tof)
    pool_size_3d_time = 4; pool_size_3d_spatial = 2; x_tof = layers.MaxPooling3D(pool_size=(pool_size_3d_time, pool_size_3d_spatial, pool_size_3d_spatial))(x_tof)

    # Calculate target shape statically for Reshape
    reduced_seq_len_static = seq_len // pool_size_3d_time if seq_len is not None else None
    reduced_dim1 = tof_shape[0] // pool_size_3d_spatial
    reduced_dim2 = tof_shape[1] // pool_size_3d_spatial
    num_filters_tof = 128
    flattened_tof_features = reduced_dim1 * reduced_dim2 * num_filters_tof
    reshape_target_tof = (reduced_seq_len_static, flattened_tof_features)
    x_tof = layers.Reshape(reshape_target_tof)(x_tof)

    # --- Fusion ---
    x_concat = layers.Concatenate(axis=-1)([x_imu, x_thm, x_tof])

    # --- GRU Layer ---
    x = layers.Bidirectional(layers.GRU(256, dropout=LSTM_DROPOUT_RATE, recurrent_dropout=LSTM_RECURRENT_DROPOUT_RATE, return_sequences=False))(x_concat)

    # --- Dense Head ---
    x = layers.Dense(256, activation='relu')(x); x = layers.Dropout(DENSE_DROPOUT_RATE)(x)
    output = layers.Dense(n_classes, activation='softmax', name='output')(x)

    model = keras.Model(inputs=[input_imu, input_thm, input_tof], outputs=output)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ==============================================================================
#                      COMPETITION METRIC FUNCTION (UNCHANGED)
# ==============================================================================
def hierarchical_macro_f1(y_true, y_pred_labels, target_gestures_list, le):
    """Calculates the CMI competition metric with corrected variable scope."""
    # Ensure inputs are not empty first
    if len(y_true) == 0 or len(y_pred_labels) == 0:
        return 0.0, 0.0, 0.0

    # --- Ensure assignments happen before potential use ---
    y_true_np = np.asarray(y_true)
    y_pred_labels_np = np.asarray(y_pred_labels)
    # --- End Assignment Fix ---

    # --- Check for valid label indices ---
    known_labels = np.arange(len(le.classes_))
    # Create mask where predicted labels are within the known range
    valid_pred_mask = np.isin(y_pred_labels_np, known_labels)

    # Filter both true and predicted labels based on the mask
    y_true_filtered = y_true_np[valid_pred_mask]
    y_pred_filtered = y_pred_labels_np[valid_pred_mask]

    # If filtering removed all samples, return 0
    if len(y_true_filtered) == 0:
        return 0.0, 0.0, 0.0
    # --- End Validity Check ---

    # --- Proceed with metric calculation using filtered arrays ---
    try:
        y_true_str = le.inverse_transform(y_true_filtered)
        y_pred_str = le.inverse_transform(y_pred_filtered)
    except ValueError as e:
        print(f"LabelEncoder Error: {e}. Check label range. True: {np.unique(y_true_filtered)}, Pred: {np.unique(y_pred_filtered)}")
        return 0.0, 0.0, 0.0 # Return 0 if inverse_transform fails

    y_true_bin = np.isin(y_true_str, target_gestures_list)
    y_pred_bin = np.isin(y_pred_str, target_gestures_list)
    binary_f1 = f1_score(y_true_bin, y_pred_bin, pos_label=True, zero_division=0)

    y_true_mc = np.where(y_true_bin, y_true_str, 'non_target')
    y_pred_mc = np.where(y_pred_bin, y_pred_str, 'non_target')
    unique_labels_mc_present = np.unique(np.concatenate((y_true_mc, y_pred_mc)))

    gesture_f1 = f1_score(y_true_mc, y_pred_mc, labels=unique_labels_mc_present, average='macro', zero_division=0)

    metric = 0.5 * binary_f1 + 0.5 * gesture_f1
    return metric, binary_f1, gesture_f1
# ==============================================================================
#                      CROSS-VALIDATION TRAINING
# ==============================================================================

print(f"\nStarting {N_SPLITS}-Fold Cross-Validation (Shallow/Wide 3-Branch with Dropout)...")
gkf = GroupKFold(n_splits=N_SPLITS)
oof_preds = np.zeros((len(y), N_CLASSES))
oof_labels = np.zeros(len(y))
oof_groups_store = np.empty(len(y), dtype=object)
oof_indices = np.arange(len(y))

all_histories = []
fold_metrics = []

for fold, (train_idx, val_idx) in enumerate(gkf.split(X_imu, y, groups)): # Use X_imu shape for split
    print(f"\n===== Fold {fold+1}/{N_SPLITS} =====")
    # Get NumPy data slices for this fold
    X_train_imu, X_val_imu = X_imu[train_idx], X_imu[val_idx]
    X_train_thm, X_val_thm = X_thm[train_idx], X_thm[val_idx]
    X_train_tof, X_val_tof = X_tof[train_idx], X_tof[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    oof_groups_store[val_idx] = groups[val_idx] # Store subject IDs for OOF
    oof_indices[val_idx] = val_idx

    # Create tf.data datasets from NumPy arrays
    train_dataset = create_tf_dataset_from_np(X_train_imu, X_train_thm, X_train_tof, y_train, BATCH_SIZE, shuffle=True, augment=True)
    val_dataset = create_tf_dataset_from_np(X_val_imu, X_val_thm, X_val_tof, y_val, BATCH_SIZE, shuffle=False, augment=False)

    keras.backend.clear_session()
    tof_input_shape_model = (TOF_GRID_DIM, TOF_GRID_DIM, TOF_SENSORS)
    try:
        model = build_shallow_wide_3branch(MAX_LENGTH, N_FEATURES_IMU, N_FEATURES_THM, tof_input_shape_model, N_CLASSES)
        if fold == 0: model.summary()
    except Exception as e:
        print(f"Error building model: {e}"); exit()

    lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=0, min_lr=1e-6)
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1, mode='min')

    print(f"Training Fold {fold+1}...")
    try:
        history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS, callbacks=[lr_scheduler, early_stopping], verbose=1)
        all_histories.append(history)
    except Exception as e:
        print(f"Error during model training for fold {fold+1}: {e}"); continue

    print("Predicting on validation set...")
    try:
        # Predict directly on the validation dataset (which yields dictionary inputs)
        fold_oof_preds = model.predict(val_dataset)
    except Exception as e:
        print(f"Error predicting on validation dataset for fold {fold+1}: {e}")
        fold_oof_preds = np.full((len(val_idx), N_CLASSES), np.nan)

    # Store predictions
    if not np.isnan(fold_oof_preds).any():
        oof_preds[val_idx] = fold_oof_preds
        oof_labels[val_idx] = np.argmax(fold_oof_preds, axis=1)
        metric, _, _ = hierarchical_macro_f1(y_val, np.argmax(fold_oof_preds, axis=1), TARGET_GESTURES, label_encoder)
    else:
        oof_labels[val_idx] = -1; metric = 0.0; print("Skipping metric due to prediction errors.")

    fold_metrics.append(metric)
    print(f"Fold {fold+1} Hierarchical F1: {metric:.4f}")

    # Save model
    try: model.save(MODEL_TMPL.format(fold=fold+1)); print(f"Model for fold {fold+1} saved.")
    except Exception as e: print(f"Error saving model for fold {fold+1}: {e}")

    # Clean up
    del model, train_dataset, val_dataset, history; gc.collect()
    del X_train_imu, X_val_imu, X_train_thm, X_val_thm, X_train_tof, X_val_tof, y_train, y_val; gc.collect()


# ==============================================================================
#                      OVERALL OOF EVALUATION
# ==============================================================================
print("\n===== Overall OOF Evaluation (Shallow/Wide 3-Branch Dropout ) =====")
valid_oof_mask_eval = oof_labels != -1
if np.sum(valid_oof_mask_eval) > 0:
    oof_metric_overall, oof_bin_f1, oof_gest_f1 = hierarchical_macro_f1(y[valid_oof_mask_eval], oof_labels.astype(int)[valid_oof_mask_eval], TARGET_GESTURES, label_encoder)
    print(f"Overall OOF Hierarchical F1 (on {np.sum(valid_oof_mask_eval)} samples): {oof_metric_overall:.4f}")
    print(f"Mean Fold Metric: {np.mean([m for m in fold_metrics if m > 0]):.4f} +/- {np.std([m for m in fold_metrics if m > 0]):.4f}")
    print("\nOOF Classification Report (Before PP):")
    print(classification_report(y[valid_oof_mask_eval], oof_labels.astype(int)[valid_oof_mask_eval], target_names=label_encoder.classes_, zero_division=0))
else: print("No valid OOF predictions available for evaluation.")


TensorFlow Version: 2.18.0
Loading pre-trained objects and prepared NumPy arrays...
Using assumed scaled missing non-IMU indicator value: 0.0000
Loaded NumPy arrays and found feature indices:
X shape: (8151, 192, 338) (Expected features: 338)
y shape: (8151,), groups shape: (8151,)
IMU features: 13, THM features: 5, TOF features: 320
Number of classes: 18

Splitting X into IMU, THM, TOF branches...
Split array shapes: X_imu=(8151, 192, 13), X_thm=(8151, 192, 5), X_tof=(8151, 192, 8, 8, 5)

Starting 2-Fold Cross-Validation (Shallow/Wide 3-Branch with Dropout)...

===== Fold 1/2 =====


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ tof_input           │ (None, 192, 8, 8, │          0 │ -                 │
│ (InputLayer)        │ 5)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ imu_input           │ (None, 192, 13)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ thm_input           │ (None, 192, 5)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_2 (Masking) │ (None, 192, 8, 8, │          0 │ tof_input[0][0]   │
│                     │ 5)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking (Masking)   │ (None, 192, 13)   │          0 │ imu_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_1 (Masking) │ (None, 192, 5)    │          0 │ thm_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d (Conv3D)     │ (None, 192, 8, 8, │     17,408 │ masking_2[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 192, 256)  │     23,552 │ masking[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 192, 128)  │      4,608 │ masking_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 192, 8, 8, │        512 │ conv3d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 192, 256)  │      1,024 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 192, 128)  │        512 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout3d   │ (None, 192, 8, 8, │          0 │ batch_normalizat… │
│ (SpatialDropout3D)  │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d   │ (None, 192, 256)  │          0 │ batch_normalizat… │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_1 │ (None, 192, 128)  │          0 │ batch_normalizat… │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d       │ (None, 48, 4, 4,  │          0 │ spatial_dropout3… │
│ (MaxPooling3D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 48, 256)   │          0 │ spatial_dropout1… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 48, 128)   │          0 │ spatial_dropout1… │
│ (MaxPooling1D)      │                   │            │                 

 Total params: 4,315,410 (16.46 MB)

 Trainable params: 4,314,386 (16.46 MB)

 Non-trainable params: 1,024 (4.00 KB)

Training Fold 1...
Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 29s 517ms/step - accuracy: 0.0992 - loss: 2.9658 - val_accuracy: 0.1629 - val_loss: 2.6044 - learning_rate: 5.0000e-04
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 14s 451ms/step - accuracy: 0.1605 - loss: 2.5635 - val_accuracy: 0.1803 - val_loss: 2.5048 - learning_rate: 5.0000e-04
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 14s 451ms/step - accuracy: 0.1756 - loss: 2.4580 - val_accuracy: 0.1884 - val_loss: 2.4901 - learning_rate: 5.0000e-04
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 14s 451ms/step - accuracy: 0.1985 - loss: 2.3675 - val_accuracy: 0.1906 - val_loss: 2.4433 - learning_rate: 5.0000e-04
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 14s 450ms/step - accuracy: 0.2260 - loss: 2.2672 - val_accuracy: 0.2102 - val_loss: 2.3452 - learning_rate: 5.0000e-04
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 14s 451ms/step - accuracy: 0.2337 - loss: 2.2183 - val_accuracy: 0.2381 - val_loss: 2.2815 - learning_rate: 5.0000e-04
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1